In [1]:
import re
from pathlib import Path
import time

In [2]:
%%time
raw_file = Path("NCL1/NCL1_INSPVAA_original.dat")
raw_file_text = raw_file.read_text() # len = 853715702
num_rows = 100000

CPU times: total: 3.16 s
Wall time: 3.88 s


In [3]:
%%time
re_pattern = re.compile(r"#INSPVAA,.*?,.*?,.*?,.*?,.*?,.*?,.*?;(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?)\*+.*\n")
all_groups = re_pattern.findall(raw_file_text)
all_groups = all_groups[:num_rows]


CPU times: total: 19.5 s
Wall time: 23.1 s


In [4]:
print(f"{len(all_groups)}\n{all_groups[:10:]}")

100000
[('2220', '400732.750000000', '45.30228109290', '-124.96596984774', '-28.3449', '-0.6858', '0.1014', '0.2413', '-3.550174056', '1.675694720', '225.485958675', 'INS_SOLUTION_GOOD'), ('2220', '400732.800000000', '45.30228080853', '-124.96596980582', '-28.3340', '-0.6380', '0.0788', '0.2734', '-3.540661065', '1.372732918', '225.573498898', 'INS_SOLUTION_GOOD'), ('2220', '400732.850000000', '45.30228054897', '-124.96596978011', '-28.3225', '-0.5833', '0.0534', '0.2926', '-3.183019898', '1.125801142', '225.649088263', 'INS_SOLUTION_GOOD'), ('2220', '400732.900000000', '45.30228030725', '-124.96596976192', '-28.3104', '-0.5440', '0.0405', '0.2950', '-2.585130504', '0.941245086', '225.704566396', 'INS_SOLUTION_GOOD'), ('2220', '400732.950000000', '45.30228008140', '-124.96596974837', '-28.2990', '-0.5122', '0.0346', '0.2801', '-1.819054504', '0.826606015', '225.732879767', 'INS_SOLUTION_GOOD'), ('2220', '400733.000000000', '45.30227986815', '-124.96596973566', '-28.2893', '-0.4850', '0

In [5]:
# %%time
# re_pattern = re.compile(r";(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?),(.*?)\*+.*\n")
# all_groups = re_pattern.findall(raw_file_text)
# print(len(all_groups), all_groups[:10])


In [6]:
import pandas as pd
import numpy as np

C:\Users\madha\AppData\Local\Temp\ipykernel_26072\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
HEADER = [
    "Message",
    "Port",
    "Sequence #",
    "% Idle Time",
    "Time Status",
    "Week",
    "Seconds",
    "Receiver Status",
    "Reserved",
    "Receiver S/W Version",
]

DATA_ENUM = (
    "Week",
    "Seconds",
    "Latitude",
    "Longitude #",
    "Height",
    "North Velocity",
    "East Velocity",
    "Up Velocity",
    "Roll",
    "Pitch",
    "Azimuth",
    "Status",
)
DATA_ENUM_DTYPES = ('int', 'float', 'float', 'float', 'float', 'float', 'float','float', 'float', 'float', 'float', 'object')

In [8]:
list(zip(DATA_ENUM, DATA_ENUM_DTYPES))

[('Week', 'int'),
 ('Seconds', 'float'),
 ('Latitude', 'float'),
 ('Longitude #', 'float'),
 ('Height', 'float'),
 ('North Velocity', 'float'),
 ('East Velocity', 'float'),
 ('Up Velocity', 'float'),
 ('Roll', 'float'),
 ('Pitch', 'float'),
 ('Azimuth', 'float'),
 ('Status', 'object')]

In [9]:
%%time
np_array = np.array(all_groups, dtype=list(zip(DATA_ENUM, DATA_ENUM_DTYPES)))
# np_array

CPU times: total: 125 ms
Wall time: 332 ms


In [10]:
%%time
df = pd.DataFrame(np_array)
df

CPU times: total: 15.6 ms
Wall time: 29 ms


,Week,Seconds,Latitude,Longitude #,Height,North Velocity,East Velocity,Up Velocity,Roll,Pitch,Azimuth,Status
0,2220,400732.75,45.302281,-124.965970,-28.3449,-0.6858,0.1014,0.2413,-3.550174,1.675695,225.485959,INS_SOLUTION_GOOD
1,2220,400732.80,45.302281,-124.965970,-28.3340,-0.6380,0.0788,0.2734,-3.540661,1.372733,225.573499,INS_SOLUTION_GOOD
2,2220,400732.85,45.302281,-124.965970,-28.3225,-0.5833,0.0534,0.2926,-3.183020,1.125801,225.649088,INS_SOLUTION_GOOD
3,2220,400732.90,45.302280,-124.965970,-28.3104,-0.5440,0.0405,0.2950,-2.585131,0.941245,225.704566,INS_SOLUTION_GOOD
4,2220,400732.95,45.302280,-124.965970,-28.2990,-0.5122,0.0346,0.2801,-1.819055,0.826606,225.732880,INS_SOLUTION_GOOD
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2220,405732.50,45.302377,-124.965207,-30.6922,0.6442,0.2210,0.5196,3.575584,-2.951333,8.888432,INS_SOLUTION_GOOD
99996,2220,405732.55,45.302377,-124.965206,-30.6728,0.6035,0.2478,0.4022,4.021954,-2.498504,9.150963,INS_SOLUTION_GOOD
99997,2220,405732.60,45.302377,-124.965206,-30.6584,0.5719,0.2664,0.3058,4.326765,-1.873248,9.389765,INS_SOLUTION_GOOD
99998,2220,405732.65,45.302378,-124.965206,-30.6480,0.5463,0.2824,0.2215,4.485340,-1.138065,9.587016,INS_SOLUTION_GOOD


In [11]:
%%time
df = pd.DataFrame(all_groups, columns=DATA_ENUM)

CPU times: total: 46.9 ms
Wall time: 129 ms


In [12]:
%%time
float_cols = ["Seconds", "Latitude", "Longitude #"]
for col in float_cols:
    df[col] = df[col].astype('float')

# df[float_cols] = df[float_cols].apply(pd.to_numeric)

CPU times: total: 31.2 ms
Wall time: 82.3 ms


In [13]:
# for idx, ele in enumerate(all_groups):
#     if '1810bc04' in ele:
#         print(idx, ele)

In [14]:
from gnatss.utilities.time import gpsws_to_time

In [15]:
%%time
df['timestamp1'] = list(map(gpsws_to_time, df["Week"], df["Seconds"]))

CPU times: total: 1min 5s
Wall time: 1min 10s


In [16]:
%%time
df['timestamp2'] = df.apply(lambda x: gpsws_to_time(week = x["Week"], seconds=x["Seconds"]), axis=1)

CPU times: total: 1min 6s
Wall time: 1min 10s


In [17]:
from astropy import units as u
from astropy.time import Time as AstroTime  # noqa
from astropy.time import TimeDelta
from functools import lru_cache

# a = u.wk
# a = (u.wk).to(u.d)
# print(f"{a=} {type(a)=}")
# a = 10*a
# print(f"{a=} {type(a)=}")

In [18]:
%%time
def week_seconds_to_timestamp(week: np.ndarray, seconds: np.ndarray):
    # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
    week_shape = week.shape
    gps_epoch = AstroTime(0, format="gps", scale="tai")

    astropy_days_in_week = (u.wk).to(u.d)

    
    num_weeks = week * np.full(week.shape, astropy_days_in_week)


    def calc_week_timedelta(x):
        return TimeDelta(x, format="jd", scale="tai")

    week_time = np.vectorize(calc_week_timedelta, cache=True)(num_weeks) + np.full(week.shape, gps_epoch)


    def calc_week_start(x):
        return AstroTime(x.strftime("%Y-%m-%d"), format="iso", scale="tt")
        
    week_start = np.vectorize(calc_week_start, cache=True)(week_time)


    def calc_sec_timedelta(x):
        return TimeDelta(x, format="sec")

    sec_timedelta = np.vectorize(calc_sec_timedelta, cache=True)(seconds)


    final_time = week_start + sec_timedelta


    return final_time


    # return final_time

df['timestamp3'] = week_seconds_to_timestamp(np_array["Week"], np_array["Seconds"])

CPU times: total: 41.9 s
Wall time: 44.2 s


In [21]:
%%time
def week_seconds_to_timestamp(week: np.ndarray, seconds: np.ndarray):
    # return AstroTime(AstroTime(0, format="gps", scale="tai") + TimeDelta((week * u.wk).to(u.d), format="jd", scale="tai").strftime("%Y-%m-%d"), format="iso", scale="tt") + TimeDelta(seconds, format="sec")
    week_shape = week.shape
    gps_epoch = AstroTime(0, format="gps", scale="tai")

    astropy_days_in_week = (u.wk).to(u.d)

    
    num_weeks = week * np.full(week.shape, astropy_days_in_week)


    @lru_cache(maxsize=8096)
    def calc_week_timedelta(x):
        return TimeDelta(x, format="jd", scale="tai")

    week_time = np.vectorize(calc_week_timedelta, cache=True)(num_weeks) + np.full(week.shape, gps_epoch)


    @lru_cache(maxsize=8096)
    def calc_week_start(x):
        return AstroTime(x.strftime("%Y-%m-%d"), format="iso", scale="tt")
        
    week_start = np.vectorize(calc_week_start, cache=True)(week_time)


    @lru_cache(maxsize=8096)
    def calc_sec_timedelta(x):
        return TimeDelta(x, format="sec")

    sec_timedelta = np.vectorize(calc_sec_timedelta, cache=True)(seconds)


    final_time = week_start + sec_timedelta


    return final_time


    # return final_time

df['timestamp4'] = week_seconds_to_timestamp(np_array["Week"], np_array["Seconds"])

CPU times: total: 19.8 s
Wall time: 20.9 s
